# SSVEP classification comparison

This notebook takes the SSVEP epochs and the no stim periods and used a classifier.

## Import libraries

In [ ]:
# Standard libraries
import numpy as np

# Custom libraries
from Functions import import_data
from Functions import data_tools
from Functions import processing

## Import and epoch data

In [ ]:
# Settings
files = [  
    "sub-P004_ses-S002_task-T1_run-002_eeg"   
]
ch_names = ["O1", "O2"] # List of channel names to import
fc = [1, 40]            # Cut-off frequencies for BW filter

# Isolate subject IDs
subject_ids = [file.split('_')[0] for file in files]
unique_subject_ids = list(set(subject_ids))

# Preallocate variables
eeg_epochs = [None] * len(files)
settings = [None] * len(files)

for f, file in enumerate(files):
    # Import data and markers
    [eeg_ts, eeg_data, eeg_fs] = import_data.read_xdf(f"Data\\{file}.xdf", picks=ch_names)  
    [marker_ts, markers] = import_data.read_xdf_unity_markers(f"Data\{file}.xdf")

    # Filter data
    eeg_filt = processing.butter_filt(eeg_data, fc, "bandpass", eeg_fs)

    # Create epochs from Unity markers
    (eeg_epochs, epoch_labels) = data_tools.epochs_from_unity_markers(
        eeg_time = eeg_ts,
        eeg_data = eeg_filt,
        marker_time = marker_ts,
        marker_data = markers
        )

    # Fix mispelled labels & create numeric code for them
    fixed_labels = data_tools.fix_labels(epoch_labels) 
    [label_dict, label_array] = data_tools.labels_to_dict_and_array(fixed_labels)
    

In [ ]:
# Trim epochs so that they can be in numpy array instead of list
trimmed_epochs = data_tools.trim_epochs(eeg_epochs)

## Riemmanian geometry classification

In [51]:
import scipy
import sklearn
import numpy as np
from pyriemann.estimation import Covariances
from pyriemann.tangentspace import TangentSpace
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix as cm
from imblearn.over_sampling import ADASYN

# Create classifier
pipelines_fb = make_pipeline(
    Covariances(estimator="lwf"),
    TangentSpace(),
    LogisticRegression(solver="lbfgs", multi_class="auto"),
    )

# Oversample stimulus classes
sm = ADASYN(
    random_state = 42,
    )
sm.n_neighbors = 3  # This is needed because if n_neighbors <= nsamples (i.e., 4) the code breaks
[nepochs, nchans, nsamples] = np.shape(trimmed_epochs)
reshaped_epochs = np.reshape(trimmed_epochs, [nepochs, nchans*nsamples])
[x_oversampled, y_oversampled] = sm.fit_resample(reshaped_epochs, label_array)

# - Reshape oversampled data to be [epoch, channel, samples]
x_oversampled_unfold = np.reshape(
    x_oversampled,
    [x_oversampled.shape[0], nchans, -1]
    )

# Split data
[X_train, X_test, y_train, y_test] = train_test_split(x_oversampled_unfold, y_oversampled)

# Test classifier
pipelines_fb.fit(X_train, y_train).score(X_test, y_test)

0.22727272727272727

In [52]:
y_pred = pipelines_fb.predict(X_test)
cm(y_test, y_pred)

array([[4, 0, 0, 2, 0],
       [0, 0, 3, 2, 0],
       [0, 0, 1, 1, 0],
       [0, 0, 1, 0, 0],
       [0, 0, 3, 5, 0]], dtype=int64)